In [ ]:
#imports
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Dense, Embedding, MaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
import keras
from keras.layers import Layer
import keras.backend as K
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from datetime import datetime

In [ ]:

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        e = K.squeeze(e, axis=-1)
        alpha = K.softmax(e)
        alpha = K.expand_dims(alpha, axis=-1)
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
class LstmModel:
    def __init__(self, max_words, embedding_vector_len, max_len):
        self.max_words = max_words
        self.embedding_vector_length = embedding_vector_len
        self.max_len = max_len
        self.model = self.get_model()

    def get_model(self):
        self.model = Sequential()

        self.model.add(Embedding(self.max_words, self.embedding_vector_length, input_length=self.max_len))
        self.model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Bidirectional(LSTM(200,return_sequences=True))) #added bidirectional
        self.model.add(Dropout(0.2))
        self.model.add(attention())
        self.model.add(Dense(2, activation='softmax'))
        self.model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

        return self.model

In [ ]:
model = LstmModel(max_len=3000,max_words=3000,embedding_vector_len=500).get_model()
model.summary()

In [ ]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.fit(np.array(X_train), np.array(Y_train), epochs=30, batch_size=32,validation_data=(np.array(X_test),np.array(Y_test)), callbacks=[tensorboard_callback,early_stop_callback])

In [ ]:
class LstmModelC:
    def __init__(self, max_words, embedding_vector_len, max_len):
        self.max_words = max_words
        self.embedding_vector_length = embedding_vector_len
        self.max_len = max_len
        self.model = self.get_model()

    def get_model(self):
        self.model = Sequential()

        self.model.add(Embedding(self.max_words, self.embedding_vector_length, input_length=self.max_len))
        self.model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        self.model.add(MaxPooling1D(pool_size=2))
        self.model.add(LSTM(200))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(2, activation='softmax'))
        self.model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

        return self.model

In [ ]:
modelC = LstmModelC(max_len=3000,max_words=3000,embedding_vector_len=500).get_model()
modelC.summary()
modelC.fit(np.array(X_train), np.array(Y_train), epochs=30, batch_size=32,validation_data=(np.array(X_test),np.array(Y_test)), callbacks=[tensorboard_callback,early_stop_callback])
